<a href="https://colab.research.google.com/github/JamieMcQuire23/ECG-Heartbeat-Categorisation-Dataset/blob/master/LSTM_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Long Short-Term Memory Network

* This notebook will provide a Pytorch implementation of a Recurrent Neural Network for the classification of ECG data.


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split

In [3]:
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

In [0]:
mit_train = pd.read_csv("/content/drive/My Drive/ECG_HeartBeat_Pytorch/Data/heartbeat/mitbih_train.csv",header=None)
mit_test = pd.read_csv("/content/drive/My Drive/ECG_HeartBeat_Pytorch/Data/heartbeat/mitbih_test.csv",header=None)

In [0]:
#convert to the correct form

train_vals = mit_train.values
X_train = train_vals[:, :-1]
Y_train = train_vals[:, -1].astype(int)

test_vals = mit_test.values
X_test = test_vals[:,:-1]
Y_test = test_vals[:,:-1].astype(int)

In [0]:
#create the training and validation set from the training data set
X_train, X_val, Y_train, Y_val = train_test_split(X_train,Y_train,test_size=0.2,random_state=seed)

# Create the Model

In [0]:
input_shape = (1,187)
num_classes = 5
batch_size = 100
learning_rate = 1e-3
n_epochs = 10

In [0]:
class LSTM_Model(nn.Module):

  def __init__(self,input_dim,hidden_dim,batch_size,output_classes=5,num_layers=3):

    super(LSTM_Model,self).__init__()

    self.input_dim = input_dim
    self.hidden_dim = hidden_dim
    self.batch_size = batch_size
    self.num_layers = num_layers

    self.lstm = nn.LSTM(self.input_dim,self.hidden_dim,self.num_layers)

    self.linear = nn.Linear(self.hidden_dim,output_classes)

  def forward(self,x):

    out, states = self.lstm(x)
    out = out.view(out.size(0), -1)
    out = self.linear(out)

    return out

In [22]:
lstm_model = LSTM_Model(input_dim=187,hidden_dim=64,batch_size=100,output_classes=5,num_layers=3)

lstm_model.cuda()

LSTM_Model(
  (lstm): LSTM(187, 64, num_layers=3)
  (linear): Linear(in_features=64, out_features=5, bias=True)
)

In [0]:
mit_train = pd.read_csv("/content/drive/My Drive/ECG_HeartBeat_Pytorch/Data/heartbeat/mitbih_train.csv",header=None)
mit_test = pd.read_csv("/content/drive/My Drive/ECG_HeartBeat_Pytorch/Data/heartbeat/mitbih_test.csv",header=None)

mit_train, mit_val = train_test_split(mit_train,test_size=0.2,random_state=seed)


train_tensor = torch.Tensor(mit_train.values)
val_tensor = torch.Tensor(mit_val.values)
test_tensor = torch.Tensor(mit_test.values)

trainloader = torch.utils.data.DataLoader(train_tensor,batch_size=batch_size,
                                          shuffle=False,num_workers=2)

validationloader = torch.utils.data.DataLoader(val_tensor,batch_size=batch_size,
                                               shuffle=False,num_workers=2)

testloader = torch.utils.data.DataLoader(test_tensor,batch_size=batch_size,
                                          shuffle=False,num_workers=2)

In [0]:
def createLossAndOptimizer(net, learning_rate=0.001):
    
    #Loss function
    loss = torch.nn.CrossEntropyLoss()
    
    #Optimizer
    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
    
    return(loss, optimizer)

In [0]:
import time
from torch.autograd import Variable

use_cuda = True

def trainNet(net, batch_size, n_epochs, learning_rate, trainloader, validationloader):
    
    #Print all of the hyperparameters of the training iteration:
    print("===== HYPERPARAMETERS =====")
    print("batch_size=", batch_size)
    print("epochs=", n_epochs)
    print("learning_rate=", learning_rate)
    print("=" * 30)
    
    n_batches = len(trainloader)
    
    #Create our loss and optimizer functions
    loss, optimizer = createLossAndOptimizer(net, learning_rate)
    
    #Time for printing
    training_start_time = time.time()
    
    #Loop for n_epochs
    for epoch in range(n_epochs):
        
        running_loss = 0.0
        print_every = n_batches // 10
        start_time = time.time()
        total_train_loss = 0
        
        for i, data in enumerate(trainloader):
            
            inputs = data[:, :-1]
            labels = data[:, -1].long()
            sizes = inputs.shape[0]
            #inputs form batch_size, features, time_steps
            inputs = inputs.reshape(sizes,1,187)

            #Wrap them in a Variable object
            inputs, labels = Variable(inputs), Variable(labels)

            if use_cuda and torch.cuda.is_available():
              inputs = inputs.cuda()
              labels = labels.cuda()
            
            #Set the parameter gradients to zero
            optimizer.zero_grad()
            
            #Forward pass, backward pass, optimize
            outputs = net(inputs)
            loss_size = loss(outputs, labels)
            loss_size.backward()
            optimizer.step()
            
            #Print statistics
            running_loss += loss_size.item()
            total_train_loss += loss_size.item()
            
            #Print every 10th batch of an epoch
            if (i + 1) % (print_every + 1) == 0:
                print("Epoch {}, {:d}% \t train_loss: {:.2f} took: {:.2f}s".format(
                        epoch+1, int(100 * (i+1) / n_batches), running_loss / print_every, time.time() - start_time))
                #Reset running loss and time
                running_loss = 0.0
                start_time = time.time()
            
        #At the end of the epoch, do a pass on the validation set
        total_val_loss = 0
        total = 0
        correct = 0
        for data in validationloader:

            inputs = data[:, :-1]
            labels = data[:, -1].long()
            sizes = inputs.shape[0]
            inputs = inputs.reshape(sizes,1,187)

            if use_cuda and torch.cuda.is_available():
              inputs = inputs.cuda()
              labels = labels.cuda()
            
            #Wrap tensors in Variables
            inputs, labels = Variable(inputs), Variable(labels)
            
            #Forward pass
            val_outputs = net(inputs)
            val_loss_size = loss(val_outputs, labels)
            total_val_loss += val_loss_size.item()

            _, predicted = torch.max(val_outputs.data,1)
            total += labels.size(0)
            correct += (predicted == labels).sum()
            
        print("Validation loss = {:.2f}".format(total_val_loss / len(validationloader)))

        print("Accuracy of the network on the validation data: %d %%" % (100 * correct/total))
        
    print("Training finished, took {:.2f}s".format(time.time() - training_start_time))

In [27]:
trainNet(lstm_model, batch_size, n_epochs, learning_rate, trainloader, validationloader)

===== HYPERPARAMETERS =====
batch_size= 100
epochs= 10
learning_rate= 0.001
Epoch 1, 10% 	 train_loss: 0.14 took: 0.95s
Epoch 1, 20% 	 train_loss: 0.14 took: 0.84s
Epoch 1, 30% 	 train_loss: 0.13 took: 0.79s
Epoch 1, 40% 	 train_loss: 0.15 took: 0.82s
Epoch 1, 50% 	 train_loss: 0.14 took: 0.92s
Epoch 1, 60% 	 train_loss: 0.13 took: 0.89s
Epoch 1, 70% 	 train_loss: 0.14 took: 0.87s
Epoch 1, 81% 	 train_loss: 0.14 took: 1.01s
Epoch 1, 91% 	 train_loss: 0.15 took: 0.90s
Validation loss = 0.14
Accuracy of the network on the validation data: 96 %
Epoch 2, 10% 	 train_loss: 0.13 took: 0.87s
Epoch 2, 20% 	 train_loss: 0.14 took: 0.91s
Epoch 2, 30% 	 train_loss: 0.12 took: 0.85s
Epoch 2, 40% 	 train_loss: 0.14 took: 0.84s
Epoch 2, 50% 	 train_loss: 0.13 took: 0.96s
Epoch 2, 60% 	 train_loss: 0.13 took: 0.80s
Epoch 2, 70% 	 train_loss: 0.14 took: 0.89s
Epoch 2, 81% 	 train_loss: 0.14 took: 0.90s
Epoch 2, 91% 	 train_loss: 0.15 took: 0.92s
Validation loss = 0.14
Accuracy of the network on the va

In [28]:
correct = 0
total = 0

for data in testloader:
  inputs = data[:, :-1]
  labels = data[:, -1].long()
  sizes = inputs.shape[0]
  inputs = inputs.reshape(sizes,1,187)

  if use_cuda and torch.cuda.is_available():
    inputs = inputs.cuda()
    labels = labels.cuda()

  outputs = lstm_model(inputs)
  _, predicted = torch.max(outputs.data,1)
  total += labels.size(0)
  correct += (predicted == labels).sum()

print("Accuracy of the network on the test data: %d %%" % (100 * correct/total))

Accuracy of the network on the test data: 95 %
